In [1]:
import saspy
sas = saspy.SASsession(cfgname='oda', )

def run(code):
    sas.submitLST(code)

SAS Connection established. Subprocess id is 14268



In [2]:
libref = 'SASHELP'
tbl = "cars"
tbl_name = f"{libref}.{tbl}"

In [3]:
cols = """
cylinders
drivetrain
enginesize
horsepower
length
mpg_city
mpg_highway
msrp
make
model
origin
type
weight
wheelbase
"""

cols = cols.strip().split('\n')
cols

['cylinders',
 'drivetrain',
 'enginesize',
 'horsepower',
 'length',
 'mpg_city',
 'mpg_highway',
 'msrp',
 'make',
 'model',
 'origin',
 'type',
 'weight',
 'wheelbase']

In [4]:
sdf = sas.sasdata(
    tbl,
    libref,
    dsopts={
        'keep' : ' '.join(cols),
    }
)
df = sdf.head()
df.columns = df.columns.str.lower()

df

,make,model,type,origin,drivetrain,msrp,enginesize,cylinders,horsepower,mpg_city,mpg_highway,weight,wheelbase,length
0,Acura,MDX,SUV,Asia,All,36945.0,3.5,6.0,265.0,17.0,23.0,4451.0,106.0,189.0
1,Acura,RSX Type S 2dr,Sedan,Asia,Front,23820.0,2.0,4.0,200.0,24.0,31.0,2778.0,101.0,172.0
2,Acura,TSX 4dr,Sedan,Asia,Front,26990.0,2.4,4.0,200.0,22.0,29.0,3230.0,105.0,183.0
3,Acura,TL 4dr,Sedan,Asia,Front,33195.0,3.2,6.0,270.0,20.0,28.0,3575.0,108.0,186.0
4,Acura,3.5 RL 4dr,Sedan,Asia,Front,43755.0,3.5,6.0,225.0,18.0,24.0,3880.0,115.0,197.0


In [5]:
df.iloc[0,:]

make             Acura
model              MDX
type               SUV
origin            Asia
drivetrain         All
msrp           36945.0
enginesize         3.5
cylinders          6.0
horsepower       265.0
mpg_city          17.0
mpg_highway       23.0
weight          4451.0
wheelbase        106.0
length           189.0
Name: 0, dtype: object

In [6]:
sdf = sas.sasdata(tbl, libref=libref)
df = sdf.columnInfo()
df = df[df['Variable'].str.lower().isin(cols)]
df

,Member,Num,Variable,Type,Len,Pos,Format,Label
0,SASHELP.CARS,9.0,Cylinders,Num,8.0,24.0,NaN,NaN
1,SASHELP.CARS,5.0,DriveTrain,Char,5.0,147.0,NaN,NaN
2,SASHELP.CARS,8.0,EngineSize,Num,8.0,16.0,NaN,Engine Size (L)
3,SASHELP.CARS,10.0,Horsepower,Num,8.0,32.0,NaN,NaN
5,SASHELP.CARS,15.0,Length,Num,8.0,72.0,NaN,Length (IN)
6,SASHELP.CARS,11.0,MPG_City,Num,8.0,40.0,NaN,MPG (City)
7,SASHELP.CARS,12.0,MPG_Highway,Num,8.0,48.0,NaN,MPG (Highway)
8,SASHELP.CARS,6.0,MSRP,Num,8.0,0.0,DOLLAR8.,NaN
9,SASHELP.CARS,1.0,Make,Char,13.0,80.0,NaN,NaN
10,SASHELP.CARS,2.0,Model,Char,40.0,93.0,NaN,NaN


In [7]:
code = f"""
proc sql;
    create view {tbl}_sub as
    select {', '.join(cols)}
    from {tbl_name}
    where 1=1;
quit;

PROC MEANS DATA={tbl}_sub;
RUN;
"""
run(code)

Variable,Label,N,Mean,Std Dev,Minimum,Maximum
Cylinders EngineSize Horsepower Length MPG_City MPG_Highway MSRP Weight Wheelbase,Engine Size (L) Length (IN) MPG (City) MPG (Highway) Weight (LBS) Wheelbase (IN),426 428 428 428 428 428 428 428 428,5.8075117 3.1967290 215.8855140 186.3621495 20.0607477 26.8434579 32774.86 3577.95 108.1542056,1.5584426 1.1085947 71.8360316 14.3579913 5.2382176 5.7412007 19431.72 758.9832146 8.3118130,3.0000000 1.3000000 73.0000000 143.0000000 10.0000000 12.0000000 10280.00 1850.00 89.0000000,12.0000000 8.3000000 500.0000000 238.0000000 60.0000000 66.0000000 192465.00 7190.00 144.0000000


In [8]:
col = "origin"

code = f"""
PROC freq DATA={tbl}_sub;
tables {col};
RUN;
"""
run(code)

Origin,Frequency,Percent,CumulativeFrequency,CumulativePercent
Asia,158,36.92,158,36.92
Europe,123,28.74,281,65.65
USA,147,34.35,428,100.00


In [9]:
col = "enginesize"

code = f"""
PROC UNIVARIATE DATA={tbl}_sub;
VAR {col};
RUN;
"""
run(code)

In [10]:
col = "type"
msmt = "msrp"

code = f"""
PROC SQL;
    SELECT {col},
           COUNT(*) AS count,
           ROUND(SUM({msmt})/1000000, 0.01) AS {msmt}_mil
    FROM {tbl}_sub
    GROUP BY 1
    order by 2 desc
    ;
QUIT;
"""
run(code)

Type,count,msrp_mil
Sedan,262,7.8
SUV,60,2.09
Sports,49,2.62
Wagon,30,0.87
Truck,24,0.6
Hybrid,3,0.06
